In [34]:
# LOADING LIBRARIES
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import numpy as np
from collections import Counter
from scipy.stats import norm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
# FUNCTIONS FOR PREPROCESSING

def drop_df_nas_columns(df):
    """
    DROPPING INPLACE COLUMNS THAT CONTAIN MORE THAN 70% OF MISSING VALUES
    @param: df -> The dataframe to check for missing values
    """ 
    missing_values_per_column = df.isnull().sum(axis = 0)
    df_size = df.shape
    nrows = df.shape[0]
    for col, value in missing_values_per_column.items():
        if value / nrows > 0.7:
            print(f'Dropping column: {col}')
            df.drop(col, inplace=True, axis=1)

In [45]:
# READING INITIAL DATASETS
df_5065 = pd.read_csv('/content/drive/MyDrive/PODS/AgeCategory/age_category_50-65_final.csv', sep=",", encoding='latin-1',low_memory=False)
df_5065.name = "Not Very Very Young People"

In [46]:
# count values in name column
print(df_5065['Source'].value_counts()['Complaints'])

98


In [47]:
df_list = [df_5065]
for df in df_list:
    print(f"Cleaning missing values for dataframe: {df.name} \n")
    print(f"{df.name} information BEFORE dropping columns containing more than 70% of missing data \n\n")
    print(f"{df.name} initial columns: {df.columns}\n")
    print(f"{df.name} initial shape: {df.shape}\n")
    print(f"{df.name} columns and their NA values: \n\n")
    print(df.isnull().sum(axis = 0))
    drop_df_nas_columns(df)
    print("\n")
    print(f"{df.name} information AFTER dropping columns containing more than 70% of missing data \n\n")
    print(f"{df.name} final columns: {df.columns}\n")
    print(f"{df.name} final shape: {df.shape}\n\n")

Cleaning missing values for dataframe: Not Very Very Young People 

Not Very Very Young People information BEFORE dropping columns containing more than 70% of missing data 


Not Very Very Young People initial columns: Index(['CustomerID', 'Timestamp', 'Activity', 'Source', 'Gender'], dtype='object')

Not Very Very Young People initial shape: (2915332, 5)

Not Very Very Young People columns and their NA values: 


CustomerID    0
Timestamp     0
Activity      0
Source        0
Gender        0
dtype: int64


Not Very Very Young People information AFTER dropping columns containing more than 70% of missing data 


Not Very Very Young People final columns: Index(['CustomerID', 'Timestamp', 'Activity', 'Source', 'Gender'], dtype='object')

Not Very Very Young People final shape: (2915332, 5)




In [48]:
df_5065.head()

,CustomerID,Timestamp,Activity,Source,Gender
0,2025826,2015-10-05 10:12:56.880000000,Page: 50plus,Clicks_Logged_In,V
1,2025826,2015-09-30 15:14:35.943000000,Page: 50plus,Clicks_Logged_In,V
2,2063574,2015-11-06 10:47:42.137000000,Page: 50plus,Clicks_Logged_In,M
3,2099262,2015-10-28 17:00:52.393000000,Page: inschrijven,Clicks_Logged_In,V
4,1626445,2015-08-27 21:03:34.143000000,Page: inschrijven,Clicks_Logged_In,M


In [49]:
# ADDING A NEW COLUMN TO df_XXYY DESCRIBING
# IF THE SOURCE IS AFTER OR BEFORE THE FIRST COMPLAINT
df_5065["Before/After"] = 1
customers = df_5065['CustomerID'].unique().tolist()
temp_logged_in = []
firstcomplaint = False
for count,customer in enumerate(customers):
    print(f"Percentage completed: {count/len(customers) * 100} %")
    customer_data = df_5065[df_5065['CustomerID']==customer]
    customer_data = customer_data.sort_values(by = 'Timestamp', ascending=True)
    firstcomplaint = False

    for i,row in customer_data.iterrows():
      if firstcomplaint == False:
        row['Before/After'] = 'Before'
        if row["Source"] == 'Complaints':
          firstcomplaint = True
      else:
        row['Before/After'] = 'After'
        
      temp_logged_in.append(row)
      
df_beforeafter = pd.DataFrame(temp_logged_in)
df_beforeafter

Se han truncado las últimas 5000 líneas del flujo de salida.
Percentage completed: 33.235411937508346 %
Percentage completed: 33.24876485512085 %
Percentage completed: 33.26211777273334 %
Percentage completed: 33.27547069034584 %
Percentage completed: 33.28882360795834 %
Percentage completed: 33.302176525570836 %
Percentage completed: 33.31552944318334 %
Percentage completed: 33.32888236079584 %
Percentage completed: 33.34223527840833 %
Percentage completed: 33.35558819602083 %
Percentage completed: 33.368941113633326 %
Percentage completed: 33.38229403124583 %
Percentage completed: 33.39564694885833 %
Percentage completed: 33.40899986647082 %
Percentage completed: 33.42235278408332 %
Percentage completed: 33.435705701695824 %
Percentage completed: 33.44905861930832 %
Percentage completed: 33.46241153692082 %
Percentage completed: 33.47576445453331 %
Percentage completed: 33.48911737214581 %
Percentage completed: 33.502470289758314 %
Percentage completed: 33.51582320737081 %
Percentage

,CustomerID,Timestamp,Activity,Source,Gender,Before/After
1373741,2025826,2015-07-02 16:59:27.967000000,Page: mijn_cv,Clicks_Logged_In,V,Before
717855,2025826,2015-07-02 17:00:13.447000000,Page: werkmap,Clicks_Logged_In,V,Before
1450646,2025826,2015-07-03 18:26:10.717000000,Page: mijn_berichten,Clicks_Logged_In,V,Before
1437629,2025826,2015-07-03 18:26:25.440000000,Page: mijn_berichten,Clicks_Logged_In,V,Before
2330092,2025826,2015-07-03 18:30:54.780000000,Page: mijn_berichten,Clicks_Logged_In,V,Before
...,...,...,...,...,...,...
2909139,2202595,2016-02-22 12:31:14.0000000,Question: What benefits will I get?,Questions,V,Before
2910723,2045436,2015-08-03 13:13:16.0000000,Question: General: When should I send the form...,Questions,V,Before
2914531,2210242,2016-02-25 09:38:53.0000000,Question: When is/are transferred my unemploym...,Questions,M,Before
2915106,2210242,2016-02-29 10:19:21.0000000,Question: When will I receive the first paymen...,Questions,M,Before


In [50]:
print(df_beforeafter['Before/After'].value_counts()['After'])

15493


In [51]:
#pd.set_option('display.max_rows', 100)
df_beforeafter

,CustomerID,Timestamp,Activity,Source,Gender,Before/After
1373741,2025826,2015-07-02 16:59:27.967000000,Page: mijn_cv,Clicks_Logged_In,V,Before
717855,2025826,2015-07-02 17:00:13.447000000,Page: werkmap,Clicks_Logged_In,V,Before
1450646,2025826,2015-07-03 18:26:10.717000000,Page: mijn_berichten,Clicks_Logged_In,V,Before
1437629,2025826,2015-07-03 18:26:25.440000000,Page: mijn_berichten,Clicks_Logged_In,V,Before
2330092,2025826,2015-07-03 18:30:54.780000000,Page: mijn_berichten,Clicks_Logged_In,V,Before
...,...,...,...,...,...,...
2909139,2202595,2016-02-22 12:31:14.0000000,Question: What benefits will I get?,Questions,V,Before
2910723,2045436,2015-08-03 13:13:16.0000000,Question: General: When should I send the form...,Questions,V,Before
2914531,2210242,2016-02-25 09:38:53.0000000,Question: When is/are transferred my unemploym...,Questions,M,Before
2915106,2210242,2016-02-29 10:19:21.0000000,Question: When will I receive the first paymen...,Questions,M,Before


In [52]:
df_beforeafter.to_csv(os.path.join('/content/drive/MyDrive/PODS', 'beforeafter5065.csv'), index=False)